# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [23]:
%pip install azureml-train-automl
%pip install azureml-widgets
%pip install azureml-pipeline
%pip install azureml-pipeline-steps
%pip install kaggle


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.webservice.aci import AciWebservice
from azureml.data import TabularDataset
from azureml.exceptions import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep, PythonScriptStep



In [25]:
%kaggle datasets download -d rezaunderfit/instagram-fake-and-real-accounts-dataset

UsageError: Line magic function `%kaggle` not found.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-instagram-users-train'
project_folder = './automl-pipeline-project'
experiment=Experiment(ws, experiment_name)


In [27]:
import pandas as pd

key = "instagram-fake-real-accounts"
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
else:
    dataset = pd.read_csv("./kaggle_data_ig_accounts/final-v1.csv")
    dataset = TabularDataset.Tabular.register_pandas_dataframe(dataframe=dataset, target=ws.get_default_datastore(), name=key, description="Kaggle dataset of Instagram accounts both real and fake.")

In [32]:
compute_cluster_name = "my-udacity-compute"

try:
    compute_cluster = ComputeTarget(ws, compute_cluster_name)
    print("Found an existing cluster, using it")
except ComputeTargetException as ex:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)
    
compute_cluster.wait_for_completion(show_output=True, timeout_in_minutes=10)

Found an existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Configuration Settings Rationale

- task: Since the dataset schema contains a label column `is_fake` with Boolean values, a classification model is the obvious choice for training. 
- `max_concurrent_iterations`: Determined by the size of the largest VM we are permitted to use for compute in the Udacity environment
- `primary_metric`: To get a baseline model for evaluation, I chose to optimize accuracy for the AutoML run. However, that metric "may not optimize as well for datasets that are small, have very large class skew (class imbalance), or when the expected metric value is very close to 0.0 or 1.0" (Microsoft Learn, "Set up AutoML training with Python º Metrics for classification scenarios")[https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-configure-auto-train-v1#metrics-for-classification-scenarios]. My dataset meets at least one and possibly two of these criteria (less than 800 records, and ~88% of them are labeled as fake). As such, after obtaining a baseline model and its performance metrics, I intend to run a second AutoML training job with weighted AUC as the primary metric, and then compare its performance to the first job's output.
- `featurization`: Intentionally left as the default value `auto`. 

In [33]:
automl_settings = {
    "max_concurrent_iterations": 3,
    "experiment_timeout_minutes": 25,
    "primary_metric": "AUC_weighted", 
    "n_cross_validations": 5
}

automl_config = AutoMLConfig(task="classification", 
                             path = project_folder,
                             compute_target=compute_cluster,
                             training_data = dataset,
                             label_column_name = "is_fake",
                             debug_log = "automl_errors.log",
                             **automl_settings)


In [35]:
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.core import PipelineParameter

automl_pipeline_param_primarymetric = PipelineParameter(name="primary_metric", default_value="accuracy")

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics')
                           )
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model')
                           )

automl_step = AutoMLStep(
    name = 'automl_module', 
    automl_config = automl_config,
    outputs = [metrics_data, model_data],
    allow_reuse = True
)



In [36]:
automl_pipeline = Pipeline(workspace=ws, description="An automated ML pipeline for training a classifier to detect fake IG accounts", steps = [automl_step])
pipeline_run = experiment.submit(automl_pipeline)

Created step automl_module [362e0584][7218b066-398f-4ca5-a99e-5dee1ffa9e37], (This step will run and generate new outputs)
Submitted PipelineRun a66cdf7e-6f8e-4374-943d-cf3392486c43
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a66cdf7e-6f8e-4374-943d-cf3392486c43?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-217182/workspaces/quick-starts-ws-217182&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


In [37]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

## Best Model



In [40]:
import pickle

best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download(".", show_progress=True)

with open(best_model_output._path_on_datastore, "rb") as f:
    best_model = pickle.load(f)
    
print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('22', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.3, gamma=0, max_depth=5, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.5625, reg_lambda=2.1875, subsample=0.7, tree_method='auto'))], verbose=False)), ('6', Pipeli

In [41]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('22', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.3, gamma=0, max_depth=5, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.5625, reg_lambda=2.1875, subsample=0.7, tree_method='auto'))], verbose=False)), ('6', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('randomforest

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
from azureml.core import Model

#Note: have been suddenly having much difficulty in recent days with azureml methods not finding the model & metric files from finished runs,
#even after having just downloaded them with e.g. best_model_output.download(). Due to this, I must use the register method on the Model
#class to register the model, though I would have greatly preferred to do it through the pipeline_run object itself.  
# registered_model = Model.register(workspace=ws, 
#                                     model_path=best_model_output._path_on_datastore,
#                                     model_name="best_model"
# )

automl_train_step = pipeline_run.find_step_run('automl_module')

if automl_train_step:
    train_step_obj = automl_train_step[0] 
    registered_model = train_step_obj.register_model(model_name="ig-aml-classifier", model_path="model_data")
    # train_step_obj.get_output_data('metrics_data').download("./outputs") # download the output to current directory
    # train_step_obj.get_output_data('model_data').download("./outputs")

In [50]:
import os

folder_name = "ig_classifier_service"
os.makedirs(f"./{folder_name}", exist_ok=True)

In [52]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script = "score_ig.py",
                                    runtime = "python",
                                    source_directory=folder_name)

deploy_config = AciWebservice.deploy_configuration(cpu_cores=1)

In [53]:
automl_service = Model.deploy(workspace=ws, name="ig-classifier-service",
                                models=[registered_model],
                                inference_config=inference_config,
                                deployment_config=deploy_config)
automl_service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-02 00:54:40+00:00 Creating Container Registry if not exists.
2022-12-02 00:54:41+00:00 Registering the environment.
2022-12-02 00:54:42+00:00 Building image.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
